In [12]:
import sk8s

In [13]:
image = sk8s.docker_build(image_name="ngs-1",
                          conda=["bwa", "gatk4", "samtools", "google-cloud-sdk"],
                          channels=["conda-forge", "bioconda"],
                          pip=["numpy", "scipy", "matplotlib", "pandas"],
                          additional_config="RUN apt-get install -y gcc python3-dev python3-setuptools && pip3 uninstall crcmod && pip3 install --no-cache-dir -U crcmod")
    
image

In [ ]:
# Create a volume to store the reference
reference_volume = sk8s.create_volume("100Gi", name="reference-volume")
reference_volume

In [ ]:
def sleep(n):
    import time
    time.sleep(n)

sk8s.run(sleep, 3600, asynchro=True, image=image)

In [ ]:
def populate_reference_volume(volume):
    import subprocess

    def run_silent(cmd):
        return subprocess.run(cmd, check=True, shell=True,
                              stdout=subprocess.PIPE,
                              stderr=None)

    #result = run_silent(f"""mkdir -p /mnt/{volume}/hg38/ && gsutil -m rsync -r gs://genomics-public-data/references/hg38/v0/ /mnt/{volume}/hg38/""")
    result = run_silent(f"""mkdir -p /mnt/{volume}/hg38/""")
    result = run_silent(f"""gsutil -m rsync -r  gs://jared-genome/ref/GRCh38/ /mnt/{volume}/hg38/""")
    return "OK"

result = sk8s.run(populate_reference_volume, reference_volume,
                  volumes=[reference_volume],
                  image=image,
                  asynchro=False)

In [ ]:
print(result)

In [ ]:
def align(fq1, fq2, output_bam):
    import subprocess
    subprocess.run(f"gsutil -m cp {fq1} ./fq1.fq", check=True, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    subprocess.run(f"gsutil -m cp {fq2} ./fq2.fq", check=True, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    subprocess.run(f"bwa mem {reference} fq1.fq fq2.fq | samtools sort > out.bam", check=True, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    subprocess.run(f"gsutil -m cp out.bam {output_bam}", check=True, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)